In [34]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PySpark_Hadoop")\
.master("local[*]")\
.config("spark.cores.max", 2) \
.config("spark.executor.cores", 1)\
.config("spark.sql.autoBroadcastJoinThreshold", "-1")\
.config("spark.sql.adaptive.enabled", False)\
.config("spark.sql.adaptive.coalescePartitions.enabled", False)\
.config("spark.sql.adaptive.skewJoin.enabled", False)\
.config("spark.sql.catalogImplementation", "hive")\
.getOrCreate()

In [33]:
spark.stop()

In [48]:
customer_data = spark.read.csv("file:///home/dominic/Desktop/pythonLearning/csvFiles/practice_Data/Interview_Preparation/Flight_Data/Customer Loyalty History.csv", header=True, inferSchema=True)
customer_data = customer_data.withColumnRenamed("Loyalty Number", "Loyalty")
customer_data.createOrReplaceTempView("customer_data")


flight_data = spark.read.csv("file:///home/dominic/Desktop/pythonLearning/csvFiles/practice_Data/Interview_Preparation/Flight_Data/Customer Flight Activity.csv", header=True, inferSchema=True)
flight_data = flight_data.withColumnRenamed("Loyalty Number", "Loyalty_Number")
flight_data.createOrReplaceTempView("flight_data")

customer_data.show(5)
flight_data.show(5)

+-------+-------+----------------+---------+-----------+------+---------+------+--------------+------------+-------+---------------+---------------+----------------+-----------------+------------------+
|Loyalty|Country|        Province|     City|Postal Code|Gender|Education|Salary|Marital Status|Loyalty Card|    CLV|Enrollment Type|Enrollment Year|Enrollment Month|Cancellation Year|Cancellation Month|
+-------+-------+----------------+---------+-----------+------+---------+------+--------------+------------+-------+---------------+---------------+----------------+-----------------+------------------+
| 480934| Canada|         Ontario|  Toronto|    M2Z 4K1|Female| Bachelor| 83236|       Married|        Star|3839.14|       Standard|           2016|               2|             NULL|              NULL|
| 549612| Canada|         Alberta| Edmonton|    T3G 6Y6|  Male|  College|  NULL|      Divorced|        Star|3839.61|       Standard|           2016|               3|             NULL|     

In [35]:
spark.conf.get("spark.sql.catalogImplementation")
spark.sql("show databases").show()
spark.sql("use default")
spark.sql("show tables").show()


+---------+
|namespace|
+---------+
|  default|
+---------+

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|  default|customer_flight_data|      false|
|  default|customer_flight_d...|      false|
+---------+--------------------+-----------+



25/07/17 00:57:18 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


In [36]:
# Load the data into a DataFrame
customer_flight_data = spark.read.table("customer_flight_data")

# Drop the column `Loyalty_Number`
customer_flight_data = customer_flight_data.drop("Loyalty_Number")
customer_flight_data.printSchema()

root
 |-- Loyalty: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Province: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Postal Code: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Marital Status: string (nullable = true)
 |-- Loyalty Card: string (nullable = true)
 |-- CLV: double (nullable = true)
 |-- Enrollment Type: string (nullable = true)
 |-- Enrollment Year: integer (nullable = true)
 |-- Enrollment Month: integer (nullable = true)
 |-- Cancellation Year: integer (nullable = true)
 |-- Cancellation Month: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Flights Booked: integer (nullable = true)
 |-- Flights with Companions: integer (nullable = true)
 |-- Total Flights: integer (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- Points Accumulated: double (nullable = 

In [37]:
from pyspark.sql.functions import sum

monthly_bookings = customer_flight_data.groupBy("Year", "Month") \
    .agg(sum("Flights Booked").alias("Total_Bookings")) \
    .orderBy("Year", "Month")

monthly_bookings.show()

+----+-----+--------------+
|Year|Month|Total_Bookings|
+----+-----+--------------+
|2017|    1|         47766|
|2017|    2|         47691|
|2017|    3|         64271|
|2017|    4|         55720|
|2017|    5|         65279|
|2017|    6|         80493|
|2017|    7|         89288|
|2017|    8|         78692|
|2017|    9|         63304|
|2017|   10|         60351|
|2017|   11|         56905|
|2017|   12|         77650|
|2018|    1|         49170|
|2018|    2|         48643|
|2018|    3|         66928|
|2018|    4|         56791|
|2018|    5|         76891|
|2018|    6|         92749|
|2018|    7|        104143|
|2018|    8|         90512|
+----+-----+--------------+
only showing top 20 rows



In [38]:
from pyspark.sql.functions import sum

totalBookings = customer_flight_data.groupBy("Loyalty") \
    .agg(sum("Flights Booked").alias("Total_Bookings"),
         sum("Distance").alias("Total_Distance"),
         sum("Points Accumulated").alias("Points_Earned"),
         sum("Points Redeemed").alias("Total_Points_Redeemed")) \
    .orderBy("Total_Bookings", ascending=True)

totalBookings = totalBookings.filter(totalBookings["Total_Bookings"] > 0)\
    .filter(totalBookings["Total_Points_Redeemed"] > 0)

print("Total Count: "+str(totalBookings.count()))

totalBookings.show()


Total Count: 11560


+-------+--------------+--------------+-------------+---------------------+
|Loyalty|Total_Bookings|Total_Distance|Points_Earned|Total_Points_Redeemed|
+-------+--------------+--------------+-------------+---------------------+
| 279883|             1|          1618|        161.0|                  349|
| 756663|             1|          2498|        249.0|                  453|
| 999891|             1|          2526|        252.0|                  414|
| 680485|             1|          1764|        176.0|                  463|
| 688731|             1|          2254|        225.0|                  590|
| 633162|             2|          3192|        319.0|                  511|
| 201055|             2|          1668|        166.0|                  484|
| 969719|             3|          1038|        103.0|                  536|
| 604436|             3|          1656|        165.0|                  350|
| 174669|             3|          1227|        122.0|                  539|
| 226804|   

In [39]:
correlation = customer_flight_data.stat.corr("Distance", "Points Accumulated")
print(f"Correlation between flight_distance and points_earned: {correlation}")


Correlation between flight_distance and points_earned: 0.9945637211531365


In [44]:
from pyspark.sql.functions import col

customer_flight_data_temp = customer_flight_data.withColumn(
    "CLV", 
    col("Points Accumulated") - col("Points Redeemed")
)
customer_flight_data_temp.printSchema()
customer_flight_data_temp.show()



root
 |-- Loyalty: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Province: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Postal Code: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Marital Status: string (nullable = true)
 |-- Loyalty Card: string (nullable = true)
 |-- CLV: double (nullable = true)
 |-- Enrollment Type: string (nullable = true)
 |-- Enrollment Year: integer (nullable = true)
 |-- Enrollment Month: integer (nullable = true)
 |-- Cancellation Year: integer (nullable = true)
 |-- Cancellation Month: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Flights Booked: integer (nullable = true)
 |-- Flights with Companions: integer (nullable = true)
 |-- Total Flights: integer (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- Points Accumulated: double (nullable = 

In [49]:
customer_flight_data_temp.groupBy("Loyalty Card") \
    .agg(
        sum("Total Flights").alias("Total_Flights")
    ).orderBy("Total_Flights", ascending=True).show()

+------------+-------------+
|Loyalty Card|Total_Flights|
+------------+-------------+
|      Aurora|       431482|
|        Nova|       706167|
|        Star|       950040|
+------------+-------------+



In [52]:
from pyspark.sql.functions import count

customer_flight_data_temp.groupBy("Loyalty Card", "gender") \
    .agg(
        count("Loyalty Card")
    ).select("Loyalty Card", "gender", "count(Loyalty Card)") \
    .orderBy("count(Loyalty Card)", ascending=False) \
    .show()

+------------+------+-------------------+
|Loyalty Card|gender|count(Loyalty Card)|
+------------+------+-------------------+
|        Star|  Male|              92376|
|        Star|Female|              91848|
|        Nova|Female|              69984|
|        Nova|  Male|              67584|
|      Aurora|  Male|              42024|
|      Aurora|Female|              41808|
+------------+------+-------------------+

